In [2]:
# pip install python_speech_features
import numpy as np
import csv
from scipy.io import wavfile as wav
from scipy.fftpack import fft
from python_speech_features import mfcc, delta, logfbank
from random import randint
import librosa


FileNames = {"train":"ml-fmi-23-2020//train.txt", "valid":"ml-fmi-23-2020//validation.txt", "test":"ml-fmi-23-2020//test.txt", "ex":"ml-fmi-23-2020//sample_submission.txt","pred":"ml-fmi-23-2020//predictions.txt"}

AudioFolders = {"train":"ml-fmi-23-2020//audio//train//", "valid":"ml-fmi-23-2020//audio//validation//", "test":"ml-fmi-23-2020//audio//test//"}

sr   = 16000 # Sample Rate   - 16 kHz
wlen = 0.025 # window length - 25 ms = 400 samples
slen = 0.01  # step   length - 10 ms = 160 samples 
nfft = 512 

def readCsv (fileName, hasLables):
    data = []
    with open(fileName, "r", newline='\n') as csvfile:
        for row in csv.reader(csvfile, delimiter=','):
            data.extend(row)
    if hasLables:
        data = np.transpose(np.array(data).reshape((len(data)//2, 2))) 
    else:
        data = np.array(data)
    return data

def writeCsv (data, labels, fileName = FileNames['pred']):
    with open(fileName, "w", newline='\n') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['name', 'label'])
        for row in np.transpose([data, labels]):
            writer.writerow(row)

def getData (folderName, dataNames):
    data = []
    for dataName in dataNames:
        audio = librosa.load(AudioFolders[folderName]+dataName)[0]
        data.append(librosa.amplitude_to_db(abs(librosa.stft(audio))))#.reshape(-1))
        # data.append(np.abs(fft(wav.read(AudioFolders[folderName]+dataName)[1])))
        # data.append(mfcc(wav.read(AudioFolders[folderName]+dataName)[1], numcep=12))#.reshape(-1))
        
    return np.array(data)

In [3]:
train_data_name, train_labels = readCsv(FileNames['train'], hasLables=True)
valid_data_name, valid_labels = readCsv(FileNames['valid'], hasLables=True)
test_data_name                = readCsv(FileNames['test'],  hasLables=False)
# writeCsv(train_data, train_labels)

In [4]:
# train_data = getData('train', train_data_name)
# valid_data = getData('valid', valid_data_name)
# test_data  = getData('test',  test_data_name)
# mi = min([train_data.min(), valid_data.min(), test_data.min()])
# ma = max([train_data.max(), valid_data.max(), test_data.max()])
# print (mi, ma)
# train = (train_data - mi) / (ma-mi)
# valid = (valid_data - mi) / (ma-mi)
# test =  (test_data  - mi) / (ma-mi)
# mi = min([train.min(), valid.min(), test.min()])
# ma = max([train.max(), valid.max(), test.max()])
# print (mi, ma)
# train.shape, valid.shape, test.shape

-57.697765 50.504
0.0 1.0


((8000, 1025, 44), (1000, 1025, 44), (3000, 1025, 44))

In [5]:
initialShape = train.shape
train = train.reshape(initialShape[0], initialShape[1], initialShape[2], 1)
initialShape = valid.shape
valid = valid.reshape(initialShape[0], initialShape[1], initialShape[2], 1)
initialShape = test.shape
test = test.reshape(initialShape[0], initialShape[1], initialShape[2], 1)
train.shape, valid.shape, test.shape

((8000, 1025, 44, 1), (1000, 1025, 44, 1), (3000, 1025, 44, 1))

In [69]:
# train_data1 = getData('train', train_data_name)
# valid_data1 = getData('valid', valid_data_name)
# test_data1 = getData('test',  test_data_name)
# mi = min([train_data1.min(), valid_data1.min(), test_data1.min()])
# ma = max([train_data1.max(), valid_data1.max(), test_data1.max()])
# print (mi, ma)
# train1 = (train_data1 - mi) / (ma-mi)
# valid1 = (valid_data1 - mi) / (ma-mi)
# test1 =  (test_data1  - mi) / (ma-mi)
# mi = min([train1.min(), valid1.min(), test1.min()])
# ma = max([train1.max(), valid1.max(), test1.max()])
# print (mi, ma)
# train1.shape, valid1.shape, test1.shape

-83.08197591063329 74.26606555409005
0.0 1.0


((8000, 99, 12), (1000, 99, 12), (3000, 99, 12))

In [ ]:
initialShape = train1.shape
train1 = train1.reshape(initialShape[0], initialShape[1], initialShape[2], 1)
initialShape = valid1.shape
valid1 = valid1.reshape(initialShape[0], initialShape[1], initialShape[2], 1)
initialShape = test.shape
test1 = test1.reshape(initialShape[0], initialShape[1], initialShape[2], 1)
train1.shape, valid1.shape, test1.shape

In [81]:
train, train1 = train1, train
valid, valid1 = valid1, valid
test,  test1  = test1,  test
train.shape, valid.shape, test.shape

((8000, 1025, 44, 1), (1000, 1025, 44, 1), (3000, 1025, 44, 1))

In [84]:
# 1
noModel = '1-stft'
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
model = Sequential()

model.add(Conv2D(16, (3,3), activation='relu', strides=(1,1), padding='same', input_shape = train[0].shape))
model.add(Conv2D(16, (3,3), activation='relu', strides=(1,1), padding='same'))
model.add(Conv2D(32, (3,3), activation='relu', strides=(1,1), padding='same'))
model.add(MaxPool2D((3,3)))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])#, 'mae', 'mse'])

model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_44 (Conv2D)           (None, 1025, 44, 16)      160       
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 1025, 44, 16)      2320      
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 1025, 44, 32)      4640      
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 341, 14, 32)       0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 152768)            0         
_________________________________________________________________
dense_41 (Dense)             (None, 32)                4888608   
_________________________________________________________________
dense_42 (Dense)             (None, 16)              

In [88]:
for epoch in range(1,15):
    model.fit(
        x = train, 
        y = np.array([[1,0] if label == '0' else [0,1] for label in train_labels]), 
        epochs = epoch+1, 
        initial_epoch = epoch,
        verbose = 1, # progress bar
        # validation_split = 0.1,
        validation_data = (valid, np.array([[1,0] if label == '0' else [0,1] for label in valid_labels])),
        shuffle = True
    )
    pred = np.array(['0' if a > b else '1' for a, b in model.predict(valid)])
    model.save('ml-fmi-23-2020//Models//' + noModel + '//' + str(epoch) + '-' + str(sum(pred==valid_labels)))

Train on 8000 samples, validate on 1000 samples
Epoch 2/2
8000/8000 [==============================] - 458s 57ms/step - loss: 0.6907 - accuracy: 0.5416 - val_loss: 0.6891 - val_accuracy: 0.4900
Train on 8000 samples, validate on 1000 samples
Epoch 3/3
8000/8000 [==============================] - 455s 57ms/step - loss: 0.6601 - accuracy: 0.6031 - val_loss: 0.5974 - val_accuracy: 0.6680
Train on 8000 samples, validate on 1000 samples
Epoch 4/4
8000/8000 [==============================] - 462s 58ms/step - loss: 0.6121 - accuracy: 0.6650 - val_loss: 0.6278 - val_accuracy: 0.6290
Train on 8000 samples, validate on 1000 samples
Epoch 5/5
8000/8000 [==============================] - 458s 57ms/step - loss: 0.5833 - accuracy: 0.6840 - val_loss: 0.5968 - val_accuracy: 0.6670
Train on 8000 samples, validate on 1000 samples
Epoch 6/6
8000/8000 [==============================] - 472s 59ms/step - loss: 0.5455 - accuracy: 0.7212 - val_loss: 0.5509 - val_accuracy: 0.7170
Train on 8000 samples, validat

In [ ]:
epoch = '4-753' # epoch-validScore
saved_model = load_model('ml-fmi-23-2020//Models//' + noModel + '//' + epoch)
pred = np.array(['0' if a > b else '1' for a, b in saved_model.predict(valid)])
sum(pred==valid_labels)

In [ ]:
pred = np.array(['0' if a > b else '1' for a, b in saved_model.predict(test)])
writeCsv(test_data_name, pred, 'ml-fmi-23-2020//Models//'+ model +'//predictions-' + model + '_' + epoch + '.txt')

In [45]:
# 2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
model = Sequential()

model.add(Conv2D(16, (5,4), activation='relu', strides=1, padding='same', input_shape = train[0].shape))
model.add(Conv2D(16, (5,4), activation='relu', strides=1, padding='same'))
model.add(MaxPool2D((5,4)))

model.add(Flatten())

model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 1025, 44, 16)      336       
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 1025, 44, 16)      5136      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 205, 11, 16)       0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 36080)             0         
_________________________________________________________________
dense_19 (Dense)             (None, 32)                1154592   
_________________________________________________________________
dense_20 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_21 (Dense)             (None, 2)                

In [46]:
for epoch in range(15):
    model.fit(
        x = train, 
        y = np.array([[1,0] if label == '0' else [0,1] for label in train_labels]), 
        epochs = 1, 
        verbose = 1, # progress bar
        # validation_split = 0.1,
        validation_data = (valid, np.array([[1,0] if label == '0' else [0,1] for label in valid_labels])),
        shuffle = True
    )
    pred = np.array(['0' if a > b else '1' for a, b in model.predict(valid)])
    model.save('ml-fmi-23-2020//Models//2//' + str(epoch) + '-' + str(sum(pred==valid_labels)))

Train on 8000 samples, validate on 1000 samples
Epoch 1/1
8000/8000 [==============================] - 378s 47ms/step - loss: 0.6939 - accuracy: 0.5061 - val_loss: 0.6925 - val_accuracy: 0.5280
Train on 8000 samples, validate on 1000 samples
Epoch 1/1
8000/8000 [==============================] - 372s 46ms/step - loss: 0.6906 - accuracy: 0.5281 - val_loss: 0.6742 - val_accuracy: 0.6420
Train on 8000 samples, validate on 1000 samples
Epoch 1/1
8000/8000 [==============================] - 361s 45ms/step - loss: 0.6616 - accuracy: 0.6084 - val_loss: 0.6721 - val_accuracy: 0.5910
Train on 8000 samples, validate on 1000 samples
Epoch 1/1
8000/8000 [==============================] - 361s 45ms/step - loss: 0.6282 - accuracy: 0.6485 - val_loss: 0.6183 - val_accuracy: 0.6670
Train on 8000 samples, validate on 1000 samples
Epoch 1/1
8000/8000 [==============================] - 361s 45ms/step - loss: 0.5956 - accuracy: 0.6775 - val_loss: 0.5856 - val_accuracy: 0.7140
Train on 8000 samples, validat

In [47]:
# 3
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
model = Sequential()

model.add(Conv2D(16, (5,4), activation='relu', strides=1, padding='same', input_shape = train[0].shape))
model.add(Conv2D(16, (5,4), activation='relu', strides=1, padding='same'))
model.add(MaxPool2D((5,4)))
model.add(Conv2D(32, (5,4), activation='relu', strides=1, padding='same'))
model.add(MaxPool2D((5,4)))

model.add(Flatten())

model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 1025, 44, 16)      336       
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 1025, 44, 16)      5136      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 205, 11, 16)       0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 205, 11, 32)       10272     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 41, 2, 32)         0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 2624)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 32)               

In [48]:
for epoch in range(15):
    model.fit(
        x = train, 
        y = np.array([[1,0] if label == '0' else [0,1] for label in train_labels]), 
        epochs = 1, 
        verbose = 1, # progress bar
        # validation_split = 0.1,
        validation_data = (valid, np.array([[1,0] if label == '0' else [0,1] for label in valid_labels])),
        shuffle = True
    )
    pred = np.array(['0' if a > b else '1' for a, b in model.predict(valid)])
    model.save('ml-fmi-23-2020//Models//3//' + str(epoch) + '-' + str(sum(pred==valid_labels)))

Train on 8000 samples, validate on 1000 samples
Epoch 1/1
8000/8000 [==============================] - 371s 46ms/step - loss: 0.6928 - accuracy: 0.5135 - val_loss: 0.6899 - val_accuracy: 0.5580
Train on 8000 samples, validate on 1000 samples
Epoch 1/1
8000/8000 [==============================] - 368s 46ms/step - loss: 0.6621 - accuracy: 0.5920 - val_loss: 0.6120 - val_accuracy: 0.6650
Train on 8000 samples, validate on 1000 samples
Epoch 1/1
8000/8000 [==============================] - 367s 46ms/step - loss: 0.6118 - accuracy: 0.6535 - val_loss: 0.5808 - val_accuracy: 0.6990
Train on 8000 samples, validate on 1000 samples
Epoch 1/1
8000/8000 [==============================] - 368s 46ms/step - loss: 0.5970 - accuracy: 0.6686 - val_loss: 0.5834 - val_accuracy: 0.6770
Train on 8000 samples, validate on 1000 samples
Epoch 1/1
8000/8000 [==============================] - 10181s 1s/step - loss: 0.5793 - accuracy: 0.6856 - val_loss: 0.5656 - val_accuracy: 0.7050
Train on 8000 samples, validat

In [ ]:
# 4
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
model = Sequential()

model.add(Conv2D(16, (5,4), activation='relu', strides=1, padding='same', input_shape = train[0].shape))
model.add(Conv2D(16, (5,4), activation='relu', strides=1, padding='same'))
model.add(MaxPool2D((5,2)))
model.add(Conv2D(32, (5,4), activation='relu', strides=1, padding='same'))
model.add(MaxPool2D((5,2)))
model.add(Conv2D(32, (3,3), activation='relu', strides=1, padding='same'))
model.add(MaxPool2D((2,2)))
model.add(Flatten())

model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [52]:
for epoch in range(15):
    model.fit(
        x = train, 
        y = np.array([[1,0] if label == '0' else [0,1] for label in train_labels]), 
        epochs = epoch+1, 
        initial_epoch = epoch,
        verbose = 1, # progress bar
        # validation_split = 0.1,
        validation_data = (valid, np.array([[1,0] if label == '0' else [0,1] for label in valid_labels])),
        shuffle = True
    )
    pred = np.array(['0' if a > b else '1' for a, b in model.predict(valid)])
    model.save('ml-fmi-23-2020//Models//4//' + str(epoch) + '-' + str(sum(pred==valid_labels)))

Train on 8000 samples, validate on 1000 samples
Epoch 1/1
8000/8000 [==============================] - 434s 54ms/step - loss: 0.4224 - accuracy: 0.7999 - val_loss: 0.5437 - val_accuracy: 0.7350
Train on 8000 samples, validate on 1000 samples
Epoch 2/2
8000/8000 [==============================] - 445s 56ms/step - loss: 0.4048 - accuracy: 0.8126 - val_loss: 0.5587 - val_accuracy: 0.7330
Train on 8000 samples, validate on 1000 samples
Epoch 3/3
8000/8000 [==============================] - 405s 51ms/step - loss: 0.3926 - accuracy: 0.8209 - val_loss: 0.5574 - val_accuracy: 0.7380
Train on 8000 samples, validate on 1000 samples
Epoch 4/4
8000/8000 [==============================] - 409s 51ms/step - loss: 0.3739 - accuracy: 0.8322 - val_loss: 0.5420 - val_accuracy: 0.7490
Train on 8000 samples, validate on 1000 samples
Epoch 5/5
8000/8000 [==============================] - 403s 50ms/step - loss: 0.3591 - accuracy: 0.8459 - val_loss: 0.5491 - val_accuracy: 0.7530
Train on 8000 samples, validat

KeyboardInterrupt: 

In [8]:
from keras.models import load_model
model = '7-stft-detailed//6'
epoch = '11-778' # epoch-validScore
saved_model = load_model('ml-fmi-23-2020//Models//' + model + '//' + epoch)
pred = np.array(['0' if a > b else '1' for a, b in saved_model.predict(valid)])
sum(pred==valid_labels)

Using TensorFlow backend.


778

In [9]:
pred = np.array(['0' if a > b else '1' for a, b in saved_model.predict(test)])
writeCsv(test_data_name, pred, 'ml-fmi-23-2020//Models//'+ model +'//predictions-' + model.replace('//', '_') + '_' + epoch + '.txt')

FileNotFoundError: [Errno 2] No such file or directory: 'ml-fmi-23-2020//Models//7-stft-detailed//6//predictions-7-stft-detailed//6_11-778.txt'

In [76]:
# # fit on both train and valid 
# train_valid = []
# train_valid.extend (train)
# train_valid.extend (valid)
# train_valid = np.array(train_valid)
# train_valid_labels = []
# train_valid_labels.extend (train_labels)
# train_valid_labels.extend (valid_labels)
# train_valid_labels = np.array(train_valid_labels)
# model.fit(
#     x = train_valid, 
#     y = np.array([[1,0] if label == '0' else [0,1] for label in train_valid_labels]), 
#     epochs = 1, 
#     verbose = 1, # progress bar
#     # validation_split = 0.1,
#     # validation_data = (valid, np.array([[1,0] if label == '0' else [0,1] for label in valid_labels])),
#     shuffle = True
# )

Train on 8100 samples, validate on 900 samples
Epoch 1/1
8100/8100 [==============================] - 17s 2ms/step - loss: 0.5013 - accuracy: 0.7659 - val_loss: 0.5724 - val_accuracy: 0.7189


In [21]:
pred1 = np.array(['0' if a > b else '1' for a, b in model.predict(test)])

In [18]:
writeCsv(test_data_name, pred)

In [22]:
sum(pred!=pred1)

383

In [ ]:
noModel = '7-stft_'

# for kaggle, comment them for other use
import os
try:
    os.mkdir('//kaggle//working//Models//')
except: pass
try:
    os.mkdir('//kaggle//working//Models//'+noModel)
except: pass
# end of for kaggle

from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
model = Sequential()

model.add(Conv2D(16, (5,4), activation='relu', strides=1, padding='same', input_shape = train[0].shape))
model.add(Conv2D(16, (5,4), activation='relu', strides=1, padding='same'))
model.add(MaxPool2D((5,2)))
model.add(Conv2D(32, (5,2), activation='relu', strides=1, padding='same'))
model.add(MaxPool2D((5,2)))
model.add(Conv2D(32, (5,2), activation='relu', strides=1, padding='same'))
model.add(MaxPool2D((2,2)))

model.add(Flatten())

model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()
epoch = -1

In [ ]:
for epoch in range(epoch+1, epoch+6+15):
    model.fit(
        x = train, 
        y = np.array([[1,0] if label == '0' else [0,1] for label in train_labels]), 
        epochs = epoch+1, 
        initial_epoch = epoch,
        verbose = 1, # progress bar
        # validation_split = 0.1,
        validation_data = (valid, np.array([[1,0] if label == '0' else [0,1] for label in valid_labels])),
        shuffle = True
    )
    pred = np.array(['0' if a > b else '1' for a, b in model.predict(valid)])
    # change where to save when not on kaggle
    model.save('//kaggle//working//Models//' + noModel + '//' + ('0' if epoch < 10 else '') + str(epoch) + '-' + str(sum(pred==valid_labels)))

In [1]:
from keras.models import load_model
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae
valid_labels_val = np.array([[1,0] if label == '0' else [0,1] for label in valid_labels])

Using TensorFlow backend.


NameError: name 'np' is not defined

In [58]:
model = '7-stft-detailed//6_14-779'
epoch1 = model[-6:] # epoch-validScore
saved_model1 = load_model('ml-fmi-23-2020//Models//' + model + '//' + epoch1)
pred1_val = saved_model1.predict(valid)
pred1 = np.array(['0' if a > b else '1' for a, b in pred1_val])
print ("sum =", sum(pred1==valid_labels), "\nmae =", mae(valid_labels_val, pred1_val), "\nmse =", mse(valid_labels_val, pred1_val))

sum = 779 
mae = 0.263045354537916 
mse = 0.1629291475414958


In [56]:
model = '7-stft-detailed//5_13-773'
epoch = model[-6:] #'13-767' # epoch-validScore
saved_model = load_model('ml-fmi-23-2020//Models//' + model + '//' + epoch)
pred_val = saved_model.predict(valid)
pred = np.array(['0' if a > b else '1' for a, b in pred_val])
print ("sum =", sum(pred==valid_labels), "\nmae =", mae(valid_labels_val, pred_val), "\nmse =", mse(valid_labels_val, pred_val))

sum = 773 
mae = 0.29725884530114755 
mse = 0.16151130637433542


In [44]:
model = '7-stft-detailed//0_13-767'
epoch = '13-767' # epoch-validScore
saved_model = load_model('ml-fmi-23-2020//Models//' + model + '//' + epoch)
pred_val = saved_model.predict(valid)
pred = np.array(['0' if a > b else '1' for a, b in pred_val])
print ("sum =", sum(pred==valid_labels), "\nmae =", mae(valid_labels_val, pred_val), "\nmse =", mse(valid_labels_val, pred_val))

sum = 767 
mae = 0.28568741813817566 
mse = 0.16552062911885457


In [63]:
model = '7-stft-detailed//6_14-779'
epoch = '11-778' # epoch-validScore
saved_model = load_model('ml-fmi-23-2020//Models//' + model + '//' + epoch)
pred_val = saved_model.predict(valid)
pred = np.array(['0' if a > b else '1' for a, b in pred_val])
print ("sum =", sum(pred==valid_labels), "\nmae =", mae(valid_labels_val, pred_val), "\nmse =", mse(valid_labels_val, pred_val))

sum = 778 
mae = 0.2971525987325676 
mse = 0.16204545836793252


In [64]:
model = '7-stft-detailed//9_11-773'
epoch = model[-6:] #'13-767' # epoch-validScore
saved_model = load_model('ml-fmi-23-2020//Models//' + model + '//' + epoch)
pred_val = saved_model.predict(valid)
pred = np.array(['0' if a > b else '1' for a, b in pred_val])
print ("sum =", sum(pred==valid_labels), "\nmae =", mae(valid_labels_val, pred_val), "\nmse =", mse(valid_labels_val, pred_val))

sum = 773 
mae = 0.3013813749161527 
mse = 0.16530229217481746


In [40]:
model = '11'
epoch4 = '20-766' # epoch-validScore
saved_model4 = load_model('ml-fmi-23-2020//Models//' + model + '//' + epoch4)
pred4_val = saved_model4.predict(valid)
pred4 = np.array(['0' if a > b else '1' for a, b in pred4_val])
print ("sum =", sum(pred4==valid_labels), "\nmae =", mae(valid_labels_val, pred4_val), "\nmse =", mse(valid_labels_val, pred4_val))

sum = 766 
mae = 0.2999433258618228 
mse = 0.17322470717130112


In [37]:
model = '3-stft_2'
epoch2 = '13-764' # epoch-validScore
saved_model2 = load_model('ml-fmi-23-2020//Models//' + model + '//' + epoch2)
pred2_val = saved_model2.predict(valid)
pred2 = np.array(['0' if a > b else '1' for a, b in pred2_val])
print ("sum =", sum(pred2==valid_labels), "\nmae =", mae(valid_labels_val, pred2_val), "\nmse =", mse(valid_labels_val, pred2_val))

sum = 764 
mae = 0.30865380359962 
mse = 0.16954820583976155


In [45]:
model = '7-stft-detailed//1_08-749'
epoch = model[-6:] #'13-767' # epoch-validScore
saved_model = load_model('ml-fmi-23-2020//Models//' + model + '//' + epoch)
pred_val = saved_model.predict(valid)
pred = np.array(['0' if a > b else '1' for a, b in pred_val])
print ("sum =", sum(pred==valid_labels), "\nmae =", mae(valid_labels_val, pred_val), "\nmse =", mse(valid_labels_val, pred_val))

sum = 749 
mae = 0.30719971271880786 
mse = 0.1740840166978262


In [38]:
model = '4'
epoch3 = '4-753' # epoch-validScore
saved_model3 = load_model('ml-fmi-23-2020//Models//' + model + '//' + epoch3)
pred3_val = saved_model3.predict(valid)
pred3 = np.array(['0' if a > b else '1' for a, b in pred3_val])
print ("sum =", sum(pred3==valid_labels), "\nmae =", mae(valid_labels_val, pred3_val), "\nmse =", mse(valid_labels_val, pred3_val))

sum = 753 
mae = 0.3016043451037472 
mse = 0.17647691567253554


In [47]:
model = '7-stft-detailed//2_20-765'
epoch = model[-6:] #'13-767' # epoch-validScore
saved_model = load_model('ml-fmi-23-2020//Models//' + model + '//' + epoch)
pred_val = saved_model.predict(valid)
pred = np.array(['0' if a > b else '1' for a, b in pred_val])
print ("sum =", sum(pred==valid_labels), "\nmae =", mae(valid_labels_val, pred_val), "\nmse =", mse(valid_labels_val, pred_val))

sum = 765 
mae = 0.3056128543322447 
mse = 0.1741405791593657


In [11]:
model = '7-stft-detailed//4_24-774'
epoch = model[-6:] #'13-767' # epoch-validScore
saved_model = load_model('ml-fmi-23-2020//Models//' + model + '//' + epoch)
pred_val = saved_model.predict(valid)
pred = np.array(['0' if a > b else '1' for a, b in pred_val])
print ("sum =", sum(pred==valid_labels), "\nmae =", mae(valid_labels_val, pred_val), "\nmse =", mse(valid_labels_val, pred_val))

sum = 774 
mae = 0.2534599614840391 
mse = 0.18167244150867268


In [10]:
pred = np.array(['0' if a > b else '1' for a, b in saved_model.predict(test)])
writeCsv(test_data_name, pred, 'ml-fmi-23-2020//Models//'+ model +'//predictions-' + model.replace('//', '_') +'txt') # + '_' + epoch + '.txt')

In [34]:
pred.shape

(3000,)